In [1]:
CLIENT_SECRETS_FILE = "client_secret.json"

In [3]:
SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl']
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'

In [12]:
import os
import pickle
import google.oauth2.credentials
 
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
 
...
...
 
def get_authenticated_service():
    credentials = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            credentials = pickle.load(token)
    #  Check if the credentials are invalid or do not exist
    if not credentials or not credentials.valid:
        # Check if the credentials have expired
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                CLIENT_SECRETS_FILE, SCOPES)
            credentials = flow.run_console()
 
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(credentials, token)
 
    return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

In [10]:
if __name__ == '__main__':
    # When running locally, disable OAuthlib's HTTPs verification. When
    # running in production *do not* leave this option enabled.
    os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'
    service = get_authenticated_service()

if __name__ == '__main__':
    # When running locally, disable OAuthlib's HTTPs verification. When
    # running in production *do not* leave this option enabled.
    os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'
    service = get_authenticated_service()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=864205785847-0uqbed7rqqd3telgp549trdtu77gkpfs.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=XubME0lvFCQL08D4ug7MzByjKvl5Jo&prompt=consent&access_type=offline
Enter the authorization code: 4/tgFiHT_9LwAc7hFIOYaPwetNvfsm_vJ8djOkNOZEeX8BsS-IgjcVMG8
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=864205785847-0uqbed7rqqd3telgp549trdtu77gkpfs.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=kJqaIqhu8eHnu0BQvmwgQwtoiOQ618&prompt=consent&access_type=offline
Enter the authorization code: 4/tgEjjBnmSeWtUK_X7D3FkT0GeSlB0MbFFSb9gXnbjzwLCrbHpaIzic8


In [15]:
#this only returns the first page of results
def search_videos_by_keyword(service, **kwargs):
    results = service.search().list(**kwargs).execute()
    for item in results['items']:
        print('%s - %s' % (item['snippet']['title'], item['id']['videoId']))
 
 
keyword = input('Enter a keyword: ')
search_videos_by_keyword(service, q=keyword, part='id,snippet', eventType='completed', type='video')
 


Enter a keyword: Tesla cybertruck
WATCH LIVE! Elon Musk presents the new Tesla Cybertruck Launch - SwvDOdBHYBw
Tesla Cybertruck live test ride - KoutN_Ezs8w
Cybertruck tesla - Teslatruck - electrictruck - SYnIg2qk_xc
Tesla Cyber Truck Unveil | 1080p60 HD | Livestream Replay - rDQ3UnPA_E8
A.I &amp; TESLA CYBERTRUCKS - THE FUTURE IS HERE - ipU8wlAM5w0


In [29]:
#get up to 3 or more pages of results
#in this case I specified 5 pages
def get_videos(service, **kwargs):
    final_results = []
    results = service.search().list(**kwargs).execute()
 
    i = 0
    max_pages = 2
    while results and i < max_pages:
        final_results.extend(results['items'])
 
        # Check if another page exists
        if 'nextPageToken' in results:
            kwargs['pageToken'] = results['nextPageToken']
            results = service.search().list(**kwargs).execute()
            i += 1
        else:
            break
 
    return final_results
 
def search_videos_by_keyword(service, **kwargs):
    results = get_videos(service, **kwargs)
    for item in results:
        print('%s - %s' % (item['snippet']['title'], item['id']['videoId']))
 
 
keyword = input('Enter a keyword: ')
search_videos_by_keyword(service, q=keyword, part='id,snippet', eventType='completed', type='video')

Enter a keyword: Tesla cybertruck
WATCH LIVE! Elon Musk presents the new Tesla Cybertruck Launch - SwvDOdBHYBw
Tesla Cybertruck live test ride - KoutN_Ezs8w
Cybertruck tesla - Teslatruck - electrictruck - SYnIg2qk_xc
Tesla Cyber Truck Unveil | 1080p60 HD | Livestream Replay - rDQ3UnPA_E8
A.I &amp; TESLA CYBERTRUCKS - THE FUTURE IS HERE - ipU8wlAM5w0
My thoughts on the Tesla cybertruck - ckWO2zFk66E
Tesla CyberTruck Review / Storm Chasing Beast for 2021 - Q1Q9PPRK_hQ
The Tesla CyberWHAT THE??? CYBERTRUCK - BEfbAXy81RU
COPPA, Tesla Cybertruck, and Disney+ The White Hatter Live Nov 24 - qb46mKJ0AHg
🔴Tesla Truck Unveil Event LIVE REWIND🔴 - gEwtrXGOW_o


In [ ]:
def get_video_comments(service, **kwargs):
    comments = []
    results = service.commentThreads().list(**kwargs).execute()
 
    while results:
        for item in results['items']:
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            comments.append(comment)
 
        # Check if another page exists
        if 'nextPageToken' in results:
            kwargs['pageToken'] = results['nextPageToken']
            results = service.commentThreads().list(**kwargs).execute()
        else:
            break
 
    return comments
print(comments)

In [34]:
import csv
 
def write_to_csv(comments):
    with open('comments.csv', 'w') as comments_file:
        comments_writer = csv.writer(comments_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        comments_writer.writerow(['Video ID', 'Title', 'Comment'])
        for row in comments:
            comments_writer.writerow(list(row))

In [ ]:
def get_videos(service, **kwargs):
    final_results = []
    results = service.search().list(**kwargs).execute()
 
    i = 0
    max_pages = 3
    while results and i < max_pages:
        final_results.extend(results['items'])
 
        # Check if another page exists
        if 'nextPageToken' in results:
            kwargs['pageToken'] = results['nextPageToken']
            results = service.search().list(**kwargs).execute()
            i += 1
        else:
            break
 
    return final_results
 
 
def search_videos_by_keyword(service, **kwargs):
    results = get_videos(service, **kwargs)
    final_result = []
    for item in results:
        title = item['snippet']['title']
        video_id = item['id']['videoId']
        comments = get_video_comments(service, part='snippet', videoId=video_id, textFormat='plainText')
        # make a tuple consisting of the video id, title, comment and add the result to 
        # the final list
        final_result.extend([(video_id, title, comment) for comment in comments]) 
 
    write_to_csv(final_result)
 